In [ ]:
import pandas as pd
import boto3
import time
import os

region = os.environ["AWS_DEFAULT_REGION"]
accessKey = os.environ["AWS_ACCESS_KEY_ID"]
secretKey = os.environ["AWS_SECRET_ACCESS_KEY"]

def get_assumeRoleCreds(accountId, roleName):
    stsClient = boto3.client(
        "sts",
        aws_access_key_id=accessKey,
        aws_secret_access_key=secretKey
    )

    roleARNString = "arn:aws:iam::{0}:role/{1}".format(accountId,roleName)
    roleSessionName = "{}-{}".format(accountId,roleName)
    response = stsClient.assume_role(
        RoleArn=roleARNString,
        RoleSessionName=roleSessionName
    )
    print ("")
    print ("Assuming role {} in child account: {}".format(roleName,accountId))
    return (response)

In [ ]:
def delete_sagemaker_notebook_instances(sagemaker):
    response = sagemaker.list_notebook_instances()
    while 'NotebookInstances' in response and response['NotebookInstances']:
        for name in [res['NotebookInstanceName'] for res in response['NotebookInstances']]:
            try:
                describe = sagemaker.describe_notebook_instance(
                    NotebookInstanceName=name
                )
            except sagemaker.exceptions.ClientError as e:
                # our notebook has probably been deleted
                print('NotebookInstance error {} -- {}', name, e)
                break

            # for in service instances stop, for stopped instances delete
            status = describe['NotebookInstanceStatus']
            print('sagemaker notebook instance', name, status)
            if status == 'InService':
                print('stopping notebook instance', name)
                sagemaker.stop_notebook_instance(
                    NotebookInstanceName=name
                )
            elif status == 'Stopped':
                print('deleting notebook instance', name)
                sagemaker.delete_notebook_instance(
                    NotebookInstanceName=name
                )
            else:
                time.sleep(1)
        # list the instances again
        response = sagemaker.list_notebook_instances()

In [ ]:
def delete_bots(bot, sleep_for=3):
    # delete bots
    response = bot.get_bots()
    if 'bots' in response and response['bots']:
        for name in [res['name'] for res in response['bots']]:
            print('found bot', name)
            aliases = bot.get_bot_aliases(botName=name)
            if 'BotAliases' in aliases and aliases['BotAliases']:
                for alias in [res['name'] for res in aliases['BotAliases']]:
                    print('delete bot alias', alias)
                    bot.delete_bot_alias(name=alias, botName=name)
                    time.sleep(sleep_for)
            bot.delete_bot(name=name)
            time.sleep(sleep_for)
    else:
        print('no bots')
    # delete intents
    response = bot.get_intents()
    if 'intents' in response and response['intents']:
        for name in [res['name'] for res in response['intents']]:
            response = bot.delete_intent(name=name)
            if response['ResponseMetadata']['HTTPStatusCode'] ==204:
                print("bot intent deleted", name)
            time.sleep(sleep_for)
    # delete slot types
    response = bot.get_slot_types()
    if 'slotTypes' in response and response['slotTypes']:
        for name in [res['name'] for res in response['slotTypes']]:
            response = bot.delete_slot_type(name=name)
            if response['ResponseMetadata']['HTTPStatusCode'] ==204:
                print("bot slot type deleted", name)
            time.sleep(sleep_for)

In [ ]:
def delete_sagemaker_endpoints(sagemaker):
    response = sagemaker.list_endpoints()
    if 'Endpoints' in response and response['Endpoints']:
        for name in [res['EndpointName'] for res in response['Endpoints']]:
            print('deleting sagemaker endpoint', name)
            response = sagemaker.delete_endpoint(
                EndpointName=name
            )
    else:
        print('no sagemaker endpoints')

In [ ]:
def delete_sagemaker_endpoint_configs(sagemaker):
    response = sagemaker.list_endpoint_configs()
    if 'EndpointConfigs' in response and response['EndpointConfigs']:
        for name in [res['EndpointConfigName'] for res in response['EndpointConfigs']]:
            print('deleting sagemaker endpoint config', name)
            response = sagemaker.delete_endpoint_config(
                EndpointConfigName=name
            )
    else:
        print('no sagemaker endpoint configs')

In [ ]:
def delete_sagemaker_models(sagemaker):
    response = sagemaker.list_models()
    if 'Models' in response and response['Models']:
        for name in [res['ModelName'] for res in response['Models']]:
            print('deleting sagemaker model', name)
            response = sagemaker.delete_model(
                ModelName=name
            )
    else:
        print('no models')

In [ ]:
def delete_rekognition_collections(rekognition):
    response = rekognition.list_collections()
    if 'CollectionIds' in response and response['CollectionIds']:
        for name in response['CollectionIds']:
            print('deleting rekognition collection', name)
            response = rekognition.delete_collection(
                CollectionId=name
            )
    else:
        print('no rekognition collections')

In [ ]:
def delete_dynamodb_tables(dynamodb):
    response = dynamodb.list_tables()
    if 'TableNames' in response and response['TableNames']:
        for name in response['TableNames']:
            print('deleting dynamodb table', name)
            response = dynamodb.delete_table(
                TableName=name
            )
    else:
        print('no dynamodb tables')    

In [ ]:
def cleanup(sts_credentials, region):
    # clean up sagemaker
    sagemaker = boto3.client('sagemaker', region,
        aws_access_key_id=sts_credentials['Credentials']['AccessKeyId'],
        aws_secret_access_key=sts_credentials['Credentials']['SecretAccessKey'],
        aws_session_token=sts_credentials['Credentials']['SessionToken']
    )
    delete_sagemaker_notebook_instances(sagemaker)
    delete_sagemaker_endpoints(sagemaker)
    delete_sagemaker_endpoint_configs(sagemaker)
    delete_sagemaker_models(sagemaker)
    
    # clean up lex bots/slots/intents
    bot = boto3.client('lex-models',
        aws_access_key_id=sts_credentials['Credentials']['AccessKeyId'],
        aws_secret_access_key=sts_credentials['Credentials']['SecretAccessKey'],
        aws_session_token=sts_credentials['Credentials']['SessionToken']
    )
    delete_bots(bot)

    # clean up rekognition
    rekognition = boto3.client('rekognition', region,
        aws_access_key_id=sts_credentials['Credentials']['AccessKeyId'],
        aws_secret_access_key=sts_credentials['Credentials']['SecretAccessKey'],
        aws_session_token=sts_credentials['Credentials']['SessionToken']
    )    
    delete_rekognition_collections(rekognition)
    
    # delete dynamodb tables
    dynamodb = boto3.client('dynamodb', region,
        aws_access_key_id=sts_credentials['Credentials']['AccessKeyId'],
        aws_secret_access_key=sts_credentials['Credentials']['SecretAccessKey'],
        aws_session_token=sts_credentials['Credentials']['SessionToken']
    )    
    delete_dynamodb_tables(dynamodb)

In [ ]:
accountListFile = "credentials.csv"

df = pd.read_csv(accountListFile, converters={'accountID': lambda x: str(x)})
df.head(1)

In [ ]:
complete = []

In [ ]:
roleName = "OrganizationAccountAccessRole"

for accountId in df['accountID']:
    if accountId in complete:
        print('skipping', accountId)
        continue
    
    # assume role
    sts_credentials = get_assumeRoleCreds(accountId, roleName)
    cleanup(sts_credentials, region)
    
    complete.append(accountId)
    print('complete', accountId)    

In [ ]:
complete